In [1]:
import pandas as pd

# 모든 열이 출력되도록 설정
pd.set_option('display.max_columns', None)

df = pd.read_csv("../data/processed/db_network_data_table.csv")

# 11.1s

## 음수가 포함된 칼럼에서 음수의 갯수 확인

In [2]:
# 각 칼럼에서 음수인 값 개수 확인
negative_column_counts = df.lt(0).sum()

# 음수가 있는 칼럼만 필터링
columns_with_negative_values = negative_column_counts[negative_column_counts > 0]

# 결과를 DataFrame으로 변환하여 칼럼명 설정
negative_columns_df = columns_with_negative_values.reset_index()
negative_columns_df.columns = ['Negative columns name', 'Negative Count']

# 출력
print(negative_columns_df)
print()
print(f"음수가 포함된 칼럼 수: {len(columns_with_negative_values)}개")

# 각 row에서 음수가 있는지 확인 (음수가 하나라도 있으면 True, 그렇지 않으면 False)
rows_with_negative_values = df.lt(0).any(axis=1)

# 음수가 포함된 row 수 계산
negative_rows_count = rows_with_negative_values.sum()

print(f"음수가 포함된 데이터 수: {negative_rows_count}")

# 전체 데이터의 개수
total_data_count = df.size

# 음수가 포함된 데이터 수 (각 row에서 음수가 포함된 값의 개수)
negative_values_count = (df < 0).sum().sum()

# 음수가 포함된 데이터의 비율
negative_values_ratio = negative_values_count / total_data_count

print(f"전체 데이터에서 음수가 포함된 데이터 수의 비율: {negative_values_ratio:.4f}")

# 0.4s

      Negative columns name  Negative Count
0             Flow Duration             115
1              Flow Bytes/s              85
2            Flow Packets/s             115
3             Flow IAT Mean             115
4              Flow IAT Max             115
5              Flow IAT Min            2891
6               Fwd IAT Min              17
7         Fwd Header Length              35
8         Bwd Header Length              22
9       Fwd Header Length.1              35
10   Init_Win_bytes_forward         1001189
11  Init_Win_bytes_backward         1441552
12     min_seg_size_forward              35

음수가 포함된 칼럼 수: 13개
음수가 포함된 데이터 수: 1444427
전체 데이터에서 음수가 포함된 데이터 수의 비율: 0.0107


## 각 칼럼에서 등장하는 음수 값 카운트

In [3]:
# 각 칼럼에서 등장하는 음수 값의 카운트
for column in columns_with_negative_values.index:
    # 음수 값만 필터링
    negative_values = df[column][df[column] < 0]
    
    # 음수 값의 등장 횟수
    negative_values_count = negative_values.value_counts()
    
    if not negative_values_count.empty:  # 음수가 하나라도 있으면 출력
        print(f"[{column}]: {negative_values_count.sum()} 개")

        # 각 음수 값에 해당하는 'Label' 칼럼의 value_counts 출력
        for value in negative_values_count.index:
            # 해당 음수 값에 해당하는 'Label' 칼럼의 value_counts 출력
            label_counts = df[df[column] == value]['Label'].value_counts()

            # 음수 값과 그 갯수, 레이블 분포를 한 줄로 출력
            label_str = ', '.join([f"{label}: {count}" for label, count in label_counts.items()])
            print(f"Value: {value}\tCount: {negative_values_count[value]}, Labels({len(label_counts)}): [{label_str}]")
        print()  # 각 칼럼별로 구분을 위해 빈 줄 추가

# 0.8s

[Flow Duration]: 115 개
Value: -1.0	Count: 109, Labels(1): [0: 109]
Value: -12.0	Count: 2, Labels(1): [0: 2]
Value: -2.0	Count: 2, Labels(1): [0: 2]
Value: -13.0	Count: 1, Labels(1): [0: 1]
Value: -4.0	Count: 1, Labels(1): [0: 1]

[Flow Bytes/s]: 85 개
Value: -12000000.0	Count: 50, Labels(1): [0: 50]
Value: -6000000.0	Count: 16, Labels(1): [0: 16]
Value: -8000000.0	Count: 13, Labels(1): [0: 13]
Value: -193000000.0	Count: 1, Labels(1): [0: 1]
Value: -666667.0	Count: 1, Labels(1): [0: 1]
Value: -461538.0	Count: 1, Labels(1): [0: 1]
Value: -4000000.0	Count: 1, Labels(1): [0: 1]
Value: -1000000.0	Count: 1, Labels(1): [0: 1]
Value: -261000000.0	Count: 1, Labels(1): [0: 1]

[Flow Packets/s]: 115 개
Value: -2000000.0	Count: 109, Labels(1): [0: 109]
Value: -166667.0	Count: 2, Labels(1): [0: 2]
Value: -1000000.0	Count: 2, Labels(1): [0: 2]
Value: -153846.0	Count: 1, Labels(1): [0: 1]
Value: -500000.0	Count: 1, Labels(1): [0: 1]

[Flow IAT Mean]: 115 개
Value: -1.0	Count: 109, Labels(1): [0: 109]
Va

In [4]:
# 음수 값이 있는 데이터의 인덱스 추출
negative_flow_duration = df[df['Flow Duration'] < 0].index

# 네 칼럼에서 음수가 포함된 데이터가 동일한지 비교
same_negative_indices = (negative_flow_duration == df[df['Flow Packets/s'] < 0].index) & \
                        (negative_flow_duration == df[df['Flow IAT Mean'] < 0].index) & \
                        (negative_flow_duration == df[df['Flow IAT Max'] < 0].index)

# 결과 출력
if same_negative_indices.all():
    print("Flow Duration, Flow Packets/s, Flow IAT Mean, Flow IAT Max 칼럼의 음수가 포함된 인덱스가 동일합니다.")
else:
    print("음수가 포함된 데이터는 네 칼럼에서 동일하지 않습니다.")

# 0.0s

Flow Duration, Flow Packets/s, Flow IAT Mean, Flow IAT Max 칼럼의 음수가 포함된 인덱스가 동일합니다.


In [5]:
# 음수가 포함된 칼럼들 추출
negative_columns = df.columns[df.min() < 0]

# 'Label' 칼럼의 unique 값이 0 하나인 음수가 포함된 칼럼 추출
columns_with_only_zero_label = [
    col for col in negative_columns
    if df[df[col] < 0]['Label'].nunique() == 1 and df[df[col] < 0]['Label'].unique()[0] == 0
]

# 결과 출력
print("Label의 unique 값이 0 하나인 음수가 포함된 칼럼들:")
print(columns_with_only_zero_label)

# 0.9s

Label의 unique 값이 0 하나인 음수가 포함된 칼럼들:
['Flow Duration', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Max', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Header Length.1', 'min_seg_size_forward']


In [6]:
# 음수 데이터가 등장하는 행 필터링
rows_with_negative_values = df[
    (df[columns_with_only_zero_label] < 0).any(axis=1)
]

# 음수 데이터가 등장하는 행의 개수 확인
negative_row_count = rows_with_negative_values.shape[0]

# 결과 출력
print(f" 음수가 포함된 칼럼 중 Label의 unique 값이 0 하나인 칼럼들의 데이터 수: {negative_row_count}개")

# 0.0s

 음수가 포함된 칼럼 중 Label의 unique 값이 0 하나인 칼럼들의 데이터 수: 150개
